### Import Required Libraries and Set Up Environment Variables

In [23]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from pandas import json_normalize
from IPython.display import display

In [24]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [25]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

query_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&begin_date={begin_date}&end_date={end_date}&sort={sort}&fl={field_list}"
# query_url

In [26]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
pages = 19
for page in range(0, pages):
    # create query with a page number
    # API results show 10 articles at a time
    # offset = page * 10

    # Build URL
    reviews_url = f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(reviews_url).json()
   
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        # reviews_data = reviews.json()
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)

        # Print the page that was just retrieved
        print(f"Page {page+1} retrieved.")
    # Handle any errors that occur.
    except:
        print(f"Page {page+1} had an error. Status code: {reviews.status_code}")
        # break
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)

Page 1 retrieved.
Page 2 retrieved.
Page 3 retrieved.
Page 4 retrieved.
Page 5 retrieved.
Page 6 retrieved.
Page 7 retrieved.
Page 8 retrieved.
Page 9 retrieved.
Page 10 retrieved.
Page 11 retrieved.
Page 12 retrieved.
Page 13 retrieved.
Page 14 retrieved.
Page 15 retrieved.
Page 16 retrieved.
Page 17 retrieved.
Page 18 retrieved.
Page 19 retrieved.


In [27]:
# Preview the first 5 results in JSON format
preview_results = json.dumps(reviews["response"]["docs"][:5], indent=4)

# Use json.dumps with argument indent=4 to format data
print(preview_results)

[
    {
        "web_url": "https://www.nytimes.com/2017/08/24/movies/tales-of-an-immoral-couple-review.html",
        "snippet": "Twenty-five years after they split, two former high school sweethearts reluctantly reunite for an awkward night.",
        "source": "The New York Times",
        "headline": {
            "main": "Review: \u2018Tales of an Immoral Couple\u2019: Love Means Having to Grow Up",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "Tales of an Immoral Couple",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "Tales of an Immoral Couple (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
 

In [28]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_list_normalized = json_normalize(reviews_list)
display(reviews_list_normalized.head())

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [29]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
reviews_list_normalized["title"] = reviews_list_normalized["headline.main"].apply(
                        lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
# Display the DataFrame
display(reviews_list_normalized.head())

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [30]:
#reviews_list_df = pd.DataFrame(reviews_list[0]["response"]["docs"])

# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
# Apply the function to each row of the DataFrame
reviews_list_normalized["keywords_extracted"] = reviews_list_normalized["keywords"].apply(extract_keywords)

reviews_list_normalized.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,keywords_extracted
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,subject: Movies;creative_works: The Attachment...
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo,"subject: Movies;persons: Kapur, Shekhar;person..."
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever,subject: Movies;creative_works: You Can Live F...
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love,subject: Movies;creative_works: A Tourist's Gu...
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children,"subject: Movies;persons: Zlotowski, Rebecca;cr..."


In [31]:
# A list of Movie titles was created in the previous step

# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = reviews_list_normalized["title"].tolist()

# Display the list
display(titles_list)

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [32]:
# Prepare The Movie Database query
# This product uses the TMDB API but is not endorsed or certified by TMDB.
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [33]:
# Create an empty list to store the results
movie_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles_list:
    
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        
        # sleep for 1 sec
        print(f"Sleeping at {request_counter} requests.")
        time.sleep(1)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    response = requests.get(url + title + tmdb_key_string).json()

    # Include a try clause to search for the full movie details.
    try:

        # Get movie id
        movie_id = response["results"][0]["id"]

        # Make a request for a the full movie details
        full_movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?{tmdb_key_string}"

        # Execute "GET" request with url
        movie_details = requests.get(full_movie_url).json()
        
        # Extract the genre names into a list
        genres = movie_details.get("genres", [])

        # Extract the spoken_languages' English name into a list
        spoken_languages = movie_details.get("spoken_languages", [])

        # Extract the production_countries' name into a list
        production_countries = movie_details.get("production_countries", [])

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        relevent_data_dict = {'title': title, 
                              "genres": genres, 
                              "spoken_languages": spoken_languages, 
                              "production_countries": production_countries}
        
        movie_list.append(relevent_data_dict)
        
        # Print out the title that was found
        print(f"'{title}' details found")
    
    # Use the except clause to print out a statement if a movie
    # is not found.
    except:
        print(f"Error: Failed to retrieve movie data for '{title}'")
       

'The Attachment Diaries' details found
Error: Failed to retrieve movie data for 'What’s Love Got to Do With It?’ Probably a Lo'
'You Can Live Forever' details found
'A Tourist’s Guide to Love' details found
'Other People’s Children' details found
'One True Loves' details found
'The Lost Weekend: A Love Story' details found
'A Thousand and One' details found
'Your Place or Mine' details found
'Love in the Time of Fentanyl' details found
'Pamela, a Love Story' details found
'In From the Side' details found
'After Love' details found
'Alcarràs' details found
'Nelly & Nadine' details found
'Lady Chatterley’s Lover' details found
'The Sound of Christmas' details found
'The Inspection' details found
'Bones and All' details found
'My Policeman' details found
'About Fate' details found
'Waiting for Bojangles' details found
'I Love My Dad' details found
'A Love Song' details found
'Alone Together' details found
'Art of Love' details found
'The Wheel' details found
'Thor: Love and Thunder' detai

In [34]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
movie_list_json = json.dumps(movie_list[:5], indent=4)
print(movie_list_json)

[
    {
        "title": "The Attachment Diaries",
        "genres": [
            {
                "id": 18,
                "name": "Drama"
            },
            {
                "id": 9648,
                "name": "Mystery"
            },
            {
                "id": 53,
                "name": "Thriller"
            },
            {
                "id": 27,
                "name": "Horror"
            }
        ],
        "spoken_languages": [
            {
                "english_name": "Spanish",
                "iso_639_1": "es",
                "name": "Espa\u00f1ol"
            }
        ],
        "production_countries": [
            {
                "iso_3166_1": "AR",
                "name": "Argentina"
            }
        ]
    },
    {
        "title": "You Can Live Forever",
        "genres": [
            {
                "id": 18,
                "name": "Drama"
            },
            {
                "id": 10749,
                "name": "Roma

In [35]:
# Convert the results to a DataFrame
movie_df = pd.DataFrame(movie_list)
display(movie_df)

,title,genres,spoken_languages,production_countries
0,The Attachment Diaries,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...","[{'english_name': 'Spanish', 'iso_639_1': 'es'...","[{'iso_3166_1': 'AR', 'name': 'Argentina'}]"
1,You Can Live Forever,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."
2,A Tourist’s Guide to Love,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
3,Other People’s Children,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'FR', 'name': 'France'}]"
4,One True Loves,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CZ', 'name': 'Czech Republic'..."
...,...,...,...,...
146,God’s Own Country,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]"
147,The Mountain Between Us,"[{'id': 18, 'name': 'Drama'}, {'id': 12, 'name...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
148,In Search of Fellini,"[{'id': 18, 'name': 'Drama'}, {'id': 12, 'name...","[{'english_name': 'English', 'iso_639_1': 'en'...",[]
149,The Last Face,"[{'id': 18, 'name': 'Drama'}]","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."


### Merge and Clean the Data for Export

In [36]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = movie_df.merge(reviews_list_normalized, on="title")
merged_df.head(5)

,title,genres,spoken_languages,production_countries,web_url,snippet,source,keywords,pub_date,word_count,...,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,keywords_extracted
0,The Attachment Diaries,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...","[{'english_name': 'Spanish', 'iso_639_1': 'es'...","[{'iso_3166_1': 'AR', 'name': 'Argentina'}]",https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,...,None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,subject: Movies;creative_works: The Attachment...
1,You Can Live Forever,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,subject: Movies;creative_works: You Can Live F...
2,A Tourist’s Guide to Love,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o...",https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,subject: Movies;creative_works: A Tourist's Gu...
3,Other People’s Children,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'FR', 'name': 'France'}]",https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,...,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,"subject: Movies;persons: Zlotowski, Rebecca;cr..."
4,One True Loves,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CZ', 'name': 'Czech Republic'...",https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,...,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,"subject: Movies;persons: Bracey, Luke (1989- )..."


In [37]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
column_names = ["genres", "spoken_languages", "production_countries", "keywords"]

# Create a list of characters to remove
char_to_remove = ["[", "{", ":", '"', "'", "]", "}"]

# Loop through the list of columns to fix
for column_name in column_names:
    
    # Convert the column to type 'str'
    merged_df[column_name] = merged_df[column_name].astype(str)

    # Loop through characters to remove
    for char in char_to_remove:
        merged_df[column_name] = merged_df[column_name].str.replace(char, "", regex=False)

# Display the fixed DataFrame
merged_df.head()

,title,genres,spoken_languages,production_countries,web_url,snippet,source,keywords,pub_date,word_count,...,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,keywords_extracted
0,The Attachment Diaries,"id 18, name Drama, id 9648, name Mystery, id 5...","english_name Spanish, iso_639_1 es, name Español","iso_3166_1 AR, name Argentina",https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"name subject, value Movies, rank 1, major N, n...",2023-05-25T11:00:03+0000,295,...,None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,subject: Movies;creative_works: The Attachment...
1,You Can Live Forever,"id 18, name Drama, id 10749, name Romance","english_name English, iso_639_1 en, name Engli...","iso_3166_1 CA, name Canada, iso_3166_1 US, nam...",https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"name subject, value Movies, rank 1, major N, n...",2023-05-04T11:00:08+0000,294,...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,subject: Movies;creative_works: You Can Live F...
2,A Tourist’s Guide to Love,"id 10749, name Romance, id 35, name Comedy","english_name English, iso_639_1 en, name Engli...","iso_3166_1 US, name United States of America",https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"name subject, value Movies, rank 1, major N, n...",2023-04-21T07:03:25+0000,276,...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,subject: Movies;creative_works: A Tourist's Gu...
3,Other People’s Children,"id 18, name Drama, id 35, name Comedy","english_name English, iso_639_1 en, name Engli...","iso_3166_1 FR, name France",https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"name subject, value Movies, rank 1, major N, n...",2023-04-20T15:35:13+0000,801,...,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,"subject: Movies;persons: Zlotowski, Rebecca;cr..."
4,One True Loves,"id 10749, name Romance, id 18, name Drama, id ...","english_name English, iso_639_1 en, name Engli...","iso_3166_1 CZ, name Czech Republic, iso_3166_1...",https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"name subject, value Movies, rank 1, major N, n...",2023-04-13T11:00:06+0000,320,...,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,"subject: Movies;persons: Bracey, Luke (1989- )..."


In [38]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=["byline.person"])
# merged_df.head(2)

In [39]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates(subset=["title", "web_url"]).reset_index(drop=True)
merged_df

,title,genres,spoken_languages,production_countries,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization,keywords_extracted
0,The Attachment Diaries,"id 18, name Drama, id 9648, name Mystery, id 5...","english_name Spanish, iso_639_1 es, name Español","iso_3166_1 AR, name Argentina",https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"name subject, value Movies, rank 1, major N, n...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None,subject: Movies;creative_works: The Attachment...
1,You Can Live Forever,"id 18, name Drama, id 10749, name Romance","english_name English, iso_639_1 en, name Engli...","iso_3166_1 CA, name Canada, iso_3166_1 US, nam...",https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"name subject, value Movies, rank 1, major N, n...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None,subject: Movies;creative_works: You Can Live F...
2,A Tourist’s Guide to Love,"id 10749, name Romance, id 35, name Comedy","english_name English, iso_639_1 en, name Engli...","iso_3166_1 US, name United States of America",https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"name subject, value Movies, rank 1, major N, n...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None,subject: Movies;creative_works: A Tourist's Gu...
3,Other People’s Children,"id 18, name Drama, id 35, name Comedy","english_name English, iso_639_1 en, name Engli...","iso_3166_1 FR, name France",https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"name subject, value Movies, rank 1, major N, n...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,None,"subject: Movies;persons: Zlotowski, Rebecca;cr..."
4,One True Loves,"id 10749, name Romance, id 18, name Drama, id ...","english_name English, iso_639_1 en, name Engli...","iso_3166_1 CZ, name Czech Republic, iso_3166_1...",https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"name subject, value Movies, rank 1, major N, n...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None,"subject: Movies;persons: Bracey, Luke (1989- )..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,God’s Own Country,"id 10749, name Romance, id 18, name Drama","english_name English, iso_639_1 en, name Engli...","iso_3166_1 GB, name United Kingdom",https://www.nytimes.com/2017/10/24/movies/gods...,This feature debut of the writer and director ...,The New York Times,"name subject, value Movies, rank 1, major N, n...",2017-10-24T11:00:24+0000,295,"Review: Love Is Wild, if Not Pretty, in ‘God’s...",None,None,"Love Is Wild, if Not Always Pretty",None,None,None,By Jeannette Catsoulis,None,subject: Movies;creative_works: God's Own Coun...
147,The Mountain Between Us,"id 18, name Drama, id 12, name Adventure, id 1...","english_name English, iso_639_1 en, name English","iso_3166_1 US, name United States of America",https://www.nytimes.com/2017/10/05/movies/the-...,Idris Elba and Kate Winslet crash into a mount...,The New York Times,"name subject, value Movies, rank 1, major N, n...",2017-10-05T11:00:05+0000,272,Review: L

In [40]:
# Export data to CSV without the index
merged_df.to_csv("Exported NYT Movie data.csv", index=False)